**TODO :**
- Recheck Augmentations
- Recheck LAB normalization
- sampler for faster convergence ?

In [1]:
# %load_ext nb_black
%load_ext autoreload
%autoreload 2

### Initialization

### Imports

In [2]:
import os
import sys
import torch
import warnings
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
from matplotlib import pyplot as plt

sys.path.append("../code/")
# os.environ['CUDA_VISIBLE_DEVICES'] = "1,0"
warnings.simplefilter("ignore", UserWarning)

In [3]:
from training.main import k_fold

from utils.logger import (
    prepare_log_folder,
    save_config,
    create_logger,
    update_overall_logs,
)

from params import *

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


### Load

In [4]:
df_info = pd.read_csv(DATA_PATH + f"HuBMAP-20-dataset_information.csv")
df_mask = pd.read_csv(DATA_PATH + "train.csv")

## Training

In [5]:
BATCH_SIZES = {
    "resnet18": 64,
    "resnet34": 32,
    "resnext50_32x4d": 32,
    "se_resnext50_32x4d": 32,
    "efficientnet-b0": 32,
    "efficientnet-b1": 32,
    "efficientnet-b2": 32,
    "efficientnet-b3": 16,
    "efficientnet-b4": 16,
    "efficientnet-b5": 16,
    "efficientnet-b6": 8,
}

In [6]:
class Config:
    """
    Parameters used for training
    """
    # General
    seed = 42
    verbose = 1

    device = "cuda" if torch.cuda.is_available() else "cpu"
    save_weights = True
    sampling_mode = 'centered' # chose between 'convhull', 'centered', 'random', 'visible'
    
    # Images
    tile_size = 512
    reduce_factor = 1
    on_spot_sampling = 0.99
    overlap_factor = 1.5

    img_dir = DATA_PATH + f"train_{tile_size}_red_{reduce_factor}"
    mask_dir = DATA_PATH + f"masks_{tile_size}_red_{reduce_factor}"

    # k-fold
    cv_column = "5fold"
    random_state = 0
    selected_folds = [0, 1, 2, 3, 4]

    # Model
    encoder = "efficientnet-b1"  # "resnet18" "resnext50_32x4d", "resnet34", "efficientnet-b5"
    decoder = "Unet"  # "Unet", "DeepLabV3Plus"
    use_bot = False
    use_fpn = False
    double_model = False
    encoder_weights = "imagenet"
    num_classes = 1

    # Training
    loss = "BCEWithLogitsLoss"  # "SoftDiceLoss" / "BCEWithLogitsLoss"  / "lovasz"
    activation = "none" if loss == "lovasz" else "sigmoid"

    optimizer = "Adam"
    
    batch_size = BATCH_SIZES[encoder]
    
    if tile_size == 512:
        batch_size = batch_size // 2
        
    if batch_size >= 32:
        epochs = 50
    elif batch_size >= 16:
        epochs = 40
    elif batch_size >= 6:
        epochs = 30
    else:
        epochs = 25
    
    iter_per_epoch = 5000
    lr = 1e-3
    swa_first_epoch = 50

    warmup_prop = 0.05
    val_bs = batch_size * 2

    first_epoch_eval = 0
    
    use_fp16 = True
    # external features : https://www.kaggle.com/baesiann/glomeruli-hubmap-external-1024x1024
    # download this and put folders in input/external_data
    # api command for download (requires kaggle api + credentials):
    # kaggle datasets download -d baesiann/glomeruli-hubmap-external-1024x1024
    use_external = None # percentage of sampling on external data (training only), None to disable
    
    #pseudo labels
    # download test set & ext pseudo labels : https://www.kaggle.com/theoviel/soft-pl-932
    # save in input/external_data/PL/
    # api command :  kaggle datasets download -d theoviel/soft-pl-932   
    # external data from : https://www.kaggle.com/iafoss/hubmap-ex
    # api command : kaggle datasets download -d iafoss/hubmap-ex
    # save to input/test/
    
    # Pseudo labels now contains external data
    # must be put in input/external/theos_tiff/
    use_pseudo_label = None # percentage adding to use_external, None to disable
    soft_labels = False

In [7]:
DEBUG = False
log_folder = None

In [8]:
if not DEBUG:
    log_folder = prepare_log_folder(LOG_PATH)
    print(f"Logging results to {log_folder}")
    config_df = save_config(Config, log_folder + "config.json")
    create_logger(directory=log_folder, name="logs.txt")

metrics = k_fold(Config, log_folder=log_folder)

Logging results to ../logs/2021-05-03/8/
Creating in-memory dataset ...
Done in 86 seconds.

-------------   Fold 1 / 5  -------------

    -> 8757105 trainable parameters
    -> Validation images : ['2f6ecfcdf', 'b2dc8411c', '4ef6695ce'] 

Epoch 01/40 	 lr=5.0e-04	 t=291s	loss=0.247	val_loss=0.036 	 dice=0.7638
Epoch 02/40 	 lr=1.0e-03	 t=295s	loss=0.087	val_loss=0.011 	 dice=0.9349
Epoch 03/40 	 lr=9.7e-04	 t=290s	loss=0.077	val_loss=0.012 	 dice=0.9412
Epoch 04/40 	 lr=9.5e-04	 t=293s	loss=0.068	val_loss=0.008 	 dice=0.9545
Epoch 05/40 	 lr=9.2e-04	 t=295s	loss=0.058	val_loss=0.008 	 dice=0.9472
Epoch 06/40 	 lr=8.9e-04	 t=297s	loss=0.055	val_loss=0.008 	 dice=0.9572
Epoch 07/40 	 lr=8.7e-04	 t=295s	loss=0.058	val_loss=0.010 	 dice=0.9474
Epoch 08/40 	 lr=8.4e-04	 t=295s	loss=0.055	val_loss=0.008 	 dice=0.9551
Epoch 09/40 	 lr=8.2e-04	 t=287s	loss=0.055	val_loss=0.007 	 dice=0.9589
Epoch 10/40 	 lr=7.9e-04	 t=302s	loss=0.051	val_loss=0.007 	 dice=0.9525
Epoch 11/40 	 lr=7.6e-04	 t=3

Epoch 23/40 	 lr=4.5e-04	 t=293s	loss=0.041	val_loss=0.020 	 dice=0.9164
Epoch 24/40 	 lr=4.2e-04	 t=297s	loss=0.040	val_loss=0.021 	 dice=0.9341
Epoch 25/40 	 lr=3.9e-04	 t=295s	loss=0.040	val_loss=0.018 	 dice=0.9312
Epoch 26/40 	 lr=3.7e-04	 t=293s	loss=0.041	val_loss=0.019 	 dice=0.9250
Epoch 27/40 	 lr=3.4e-04	 t=293s	loss=0.041	val_loss=0.019 	 dice=0.9281
Epoch 28/40 	 lr=3.2e-04	 t=288s	loss=0.039	val_loss=0.018 	 dice=0.9339
Epoch 29/40 	 lr=2.9e-04	 t=292s	loss=0.037	val_loss=0.019 	 dice=0.9324
Epoch 30/40 	 lr=2.6e-04	 t=281s	loss=0.039	val_loss=0.019 	 dice=0.9297
Epoch 31/40 	 lr=2.4e-04	 t=288s	loss=0.038	val_loss=0.017 	 dice=0.9225
Epoch 32/40 	 lr=2.1e-04	 t=295s	loss=0.038	val_loss=0.018 	 dice=0.9324
Epoch 33/40 	 lr=1.8e-04	 t=300s	loss=0.037	val_loss=0.017 	 dice=0.9370
Epoch 34/40 	 lr=1.6e-04	 t=296s	loss=0.038	val_loss=0.017 	 dice=0.9353
Epoch 35/40 	 lr=1.3e-04	 t=296s	loss=0.036	val_loss=0.018 	 dice=0.9296
Epoch 36/40 	 lr=1.1e-04	 t=292s	loss=0.035	val_los

/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


  ->  Dice CV : nan  +/- nan
